In [1]:
from transformers import BertTokenizer, BertModel
import torch
import json
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open('/kaggle/input/nlp-ass4-dataset/train_file.json', 'r') as train_file:
    train_data = json.load(train_file)

with open('/kaggle/input/nlp-ass4-dataset/val_file.json', 'r') as val_file:
    val_data = json.load(val_file)

train_texts = [item["utterances"] for item in train_data]
train_labels = [item["emotions"] for item in train_data]

val_texts = [item["utterances"] for item in val_data]
val_labels = [item["emotions"] for item in val_data]

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model = bert_model.to(device)

In [26]:
labels_dict = {'anger'  :1,
               'joy'    :2,
               'fear'   :3,
               'disgust':4,
               'neutral':5,
               'surprise':6,
               'sadness':7
              }

inverse_dict = {}
for i in labels_dict:
    inverse_dict[labels_dict[i]] = i
inverse_dict[0] = 'neutral'

In [5]:
converted_inputs = []
converted_labels = []
maxs = 0
for i in train_labels:
    temp = []
    for j in i:
        temp.append(labels_dict[j])
    if(len(temp)>maxs):
        maxs = len(temp)
    converted_labels.append(temp)
print(maxs)

for i, j in enumerate(train_texts):
    temp_len = len(train_texts[i])
    if(temp_len<25):
        for lol in range(25-temp_len):
            train_texts[i].append("")
            converted_labels[i].append(0)
            
    inputs = tokenizer(train_texts[i], return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = bert_model(**inputs)
    converted_inputs.append(outputs[1])

24


In [8]:
converted_val_inputs = []
converted_val_labels = []

for i in val_labels:
    temp = []
    for j in i:
        temp.append(labels_dict[j])
    converted_val_labels.append(temp)

for i, j in enumerate(val_texts):
    temp_len = len(val_texts[i])
    if(temp_len<25):
        for lol in range(25-temp_len):
            val_texts[i].append("")
            converted_val_labels[i].append(0)
            
    inputs = tokenizer(val_texts[i], return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = bert_model(**inputs)
    converted_val_inputs.append(outputs[1])

In [19]:
for i in range(len(converted_val_labels)):
    temp_len = len(converted_val_labels[i])
    if(temp_len<25):
        for lol in range(25-temp_len):
            converted_val_labels[i].append(0) 

In [10]:
print(val_texts[0])

['Hey! Hold on a minute, hold on a second. Do you think these pearls are nice?', "I'd really prefer a mountain bike.", "Janice's birthday is coming up, I want to get her something speacial. Come in here with me.", 'Whoa, whoa, whoa, wait, whoa.', 'Do you ah, want to get her something speacial, get her flowers, get her candy, get her gum, girls love gum.', "That's a good idea, \x91Dear Janice have a Hubba-Bubba birthday'. I would like to get her something serious.", 'Oh, you want something serious.', "Y'know what you should do, you should get her one of those um, barium enemas.", 'Those are dead serious.', "All right. Look, I'm gonna go in here, and you don't buy me anything ever.", "No, no, you can't, you can't, okay, you can't, you can't buy her pearls, you just can't, you can't, you can't.", 'Why not?!', "Oh God. Uh, okay, here's the thing, this is the thing, okay, the thing is...", 'What is the thing?', "Okay. I went down to the \x91Mattress King' showroom and, and I saw Janice, kis

In [28]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size//2)
        self.fc2 = nn.Linear(hidden_size//2, hidden_size//4)
        self.fc3 = nn.Linear(hidden_size//4, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc1(out)
        out = torch.relu(out)
        out = self.fc2(out)
        out = torch.relu(out)
        out = self.fc3(out)
        return out

input_size = 768
hidden_size = 256
output_size = 8

model = GRUModel(input_size, hidden_size, output_size)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [29]:
num_epochs = 10
for epoch in range(num_epochs):
    
    model.train()
    running_loss = 0.0
    STEPS = 0
    for i, j in enumerate(train_texts):
        
        inputs = converted_inputs[i]
        labels = converted_labels[i]
        output = model(inputs.to(device))
        loss = criterion(output, torch.tensor(labels).to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        STEPS += 1

    model.eval()
    epoch_val_loss = 0.0
    VALSTEPS=0
        
    for i in range(len(val_texts)):
        VALSTEPS+=1
        inputs = converted_val_inputs[i]
        labels = converted_val_labels[i]
        output = model(inputs.to(device))
        loss = criterion(output, torch.tensor(labels).to(device))
        epoch_val_loss += loss.item()

    print('EPOCH: ' + str(epoch))
    print('AVG EPOCH TRAIN LOSS: ' + str(running_loss / STEPS))
    print('AVG EPOCH VAL LOSS: ' + str(epoch_val_loss / VALSTEPS))    

EPOCH: 0
AVG EPOCH TRAIN LOSS: 0.5036368758439774
AVG EPOCH VAL LOSS: 0.46076495437700554
EPOCH: 1
AVG EPOCH TRAIN LOSS: 0.4694363002764829
AVG EPOCH VAL LOSS: 0.44911795432545676
EPOCH: 2
AVG EPOCH TRAIN LOSS: 0.45175518237430196
AVG EPOCH VAL LOSS: 0.44741137288643373
EPOCH: 3
AVG EPOCH TRAIN LOSS: 0.4416545779172066
AVG EPOCH VAL LOSS: 0.4490980849041209
EPOCH: 4
AVG EPOCH TRAIN LOSS: 0.43662230643631683
AVG EPOCH VAL LOSS: 0.45960542567136164
EPOCH: 5
AVG EPOCH TRAIN LOSS: 0.4313927185488865
AVG EPOCH VAL LOSS: 0.434631502311036
EPOCH: 6
AVG EPOCH TRAIN LOSS: 0.4262013808832413
AVG EPOCH VAL LOSS: 0.43215312674411815
EPOCH: 7
AVG EPOCH TRAIN LOSS: 0.4225474782397926
AVG EPOCH VAL LOSS: 0.42155231570787416
EPOCH: 8
AVG EPOCH TRAIN LOSS: 0.4157249820745448
AVG EPOCH VAL LOSS: 0.42727410737182037
EPOCH: 9
AVG EPOCH TRAIN LOSS: 0.4177667418196355
AVG EPOCH VAL LOSS: 0.42823361102071805


In [30]:
torch.save(model.state_dict(), 'gru.pth')

In [27]:
model.eval()
text = ["You-you\u0085you had sex with Ursula?!","Uh, a little bit. She-she-she walked in and I thought she was you and I kissed her and","You didn't notice she was wearing different clothes?!","Well I was just so excited to see you.","Oh. Ew! Ew! Ew! Ugh! Y'know what? This is too weird."]
temp_len = len(text)
if(temp_len<25):
    for lol in range(25-temp_len):
        text.append("")

inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs = {key: val.to(device) for key, val in inputs.items()}
with torch.no_grad():
    outputs = bert_model(**inputs)

output = model(outputs[1])
final_result = []
for i in range(temp_len):
    x = output[i]
    probabilities = torch.nn.functional.softmax(x, dim=0)
    predicted_class = torch.argmax(probabilities).item()
    final_result.append(inverse_dict[predicted_class])
print(final_result)

['surprise', 'sadness', 'surprise', 'joy', 'surprise']


In [33]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size//2)
        self.fc2 = nn.Linear(hidden_size//2, hidden_size//4)
        self.fc3 = nn.Linear(hidden_size//4, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc1(out)
        out = torch.relu(out)
        out = self.fc2(out)
        out = torch.relu(out)
        out = self.fc3(out)
        return out

input_size = 768
hidden_size = 256
output_size = 8

model = LSTMModel(input_size, hidden_size, output_size)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [34]:
num_epochs = 10
for epoch in range(num_epochs):
    
    model.train()
    running_loss = 0.0
    STEPS = 0
    for i, j in enumerate(train_texts):
        
        inputs = converted_inputs[i]
        labels = converted_labels[i]
        output = model(inputs.to(device))
        loss = criterion(output, torch.tensor(labels).to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        STEPS += 1

    model.eval()
    epoch_val_loss = 0.0
    VALSTEPS=0
        
    for i in range(len(val_texts)):
        VALSTEPS+=1
        inputs = converted_val_inputs[i]
        labels = converted_val_labels[i]
        output = model(inputs.to(device))
        loss = criterion(output, torch.tensor(labels).to(device))
        epoch_val_loss += loss.item()

    print('EPOCH: ' + str(epoch))
    print('AVG EPOCH TRAIN LOSS: ' + str(running_loss / STEPS))
    print('AVG EPOCH VAL LOSS: ' + str(epoch_val_loss / VALSTEPS))    

EPOCH: 0
AVG EPOCH TRAIN LOSS: 0.4957152132411938
AVG EPOCH VAL LOSS: 0.4570693851604368
EPOCH: 1
AVG EPOCH TRAIN LOSS: 0.4498880396767047
AVG EPOCH VAL LOSS: 0.4333212915940535
EPOCH: 2
AVG EPOCH TRAIN LOSS: 0.43240580210339236
AVG EPOCH VAL LOSS: 0.42366430631585744
EPOCH: 3
AVG EPOCH TRAIN LOSS: 0.4204405360393992
AVG EPOCH VAL LOSS: 0.4269662582705489
EPOCH: 4
AVG EPOCH TRAIN LOSS: 0.4093727253170127
AVG EPOCH VAL LOSS: 0.4125832815313201
EPOCH: 5
AVG EPOCH TRAIN LOSS: 0.40240817540404356
AVG EPOCH VAL LOSS: 0.40418756675319845
EPOCH: 6
AVG EPOCH TRAIN LOSS: 0.3922956248523028
AVG EPOCH VAL LOSS: 0.4149586429304641
EPOCH: 7
AVG EPOCH TRAIN LOSS: 0.38045139784981696
AVG EPOCH VAL LOSS: 0.39061763578438125
EPOCH: 8
AVG EPOCH TRAIN LOSS: 0.366980002715586
AVG EPOCH VAL LOSS: 0.3671956500137934
EPOCH: 9
AVG EPOCH TRAIN LOSS: 0.3542145614899155
AVG EPOCH VAL LOSS: 0.37201506055920236


In [35]:
model.eval()
text = ["You-you\u0085you had sex with Ursula?!","Uh, a little bit. She-she-she walked in and I thought she was you and I kissed her and","You didn't notice she was wearing different clothes?!","Well I was just so excited to see you.","Oh. Ew! Ew! Ew! Ugh! Y'know what? This is too weird."]
temp_len = len(text)
if(temp_len<25):
    for lol in range(25-temp_len):
        text.append("")

inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs = {key: val.to(device) for key, val in inputs.items()}
with torch.no_grad():
    outputs = bert_model(**inputs)

output = model(outputs[1])
final_result = []
for i in range(temp_len):
    x = output[i]
    probabilities = torch.nn.functional.softmax(x, dim=0)
    predicted_class = torch.argmax(probabilities).item()
    final_result.append(inverse_dict[predicted_class])
print(final_result)

['surprise', 'anger', 'surprise', 'joy', 'surprise']


In [39]:
for i in range(len(final_result)-1):
    if(final_result[i]!=final_result[i+1]):
        print("Flip Detected at index: ", i)
    

Flip Detected at index:  0
Flip Detected at index:  1
Flip Detected at index:  2
Flip Detected at index:  3
